# Data Scraping of Match Reports

## Import Libraries

In [1]:
#from __future__ import print_function
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import json
import time
import sys
import logging

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### URLs of Match Reports

In [7]:
game_urls =[
    "https://fbref.com/en/matches/21b58926/Liverpool-Leeds-United-September-12-2020-Premier-League",
    "https://fbref.com/en/matches/583c2b60/Leeds-United-Fulham-September-19-2020-Premier-League",
    "https://fbref.com/en/matches/51fb894e/Sheffield-United-Leeds-United-September-27-2020-Premier-League",
    "https://fbref.com/en/matches/5ce15b58/Leeds-United-Manchester-City-October-3-2020-Premier-League",
    "https://fbref.com/en/matches/6a2c4518/Leeds-United-Wolverhampton-Wanderers-October-19-2020-Premier-League",
    "https://fbref.com/en/matches/873ab775/Aston-Villa-Leeds-United-October-23-2020-Premier-League",
    "https://fbref.com/en/matches/55d775ec/Leeds-United-Leicester-City-November-2-2020-Premier-League",
    "https://fbref.com/en/matches/527aee54/Crystal-Palace-Leeds-United-November-7-2020-Premier-League",
    "https://fbref.com/en/matches/0dd6b39b/Leeds-United-Arsenal-November-22-2020-Premier-League",
    "https://fbref.com/en/matches/3eeb39c5/Everton-Leeds-United-November-28-2020-Premier-League",
    "https://fbref.com/en/matches/419abbc0/Chelsea-Leeds-United-December-5-2020-Premier-League"]

In [16]:
play_stats_all_games = pd.DataFrame()

### Iterate through pages, collect data from different tables and combine into one dataframe

In [17]:
for i in range(len(game_urls)):
    page = pd.read_html(game_urls[i])
    if len(page) == 17:
        home_team = page[2].columns[0][0]
        away_team = page[2].columns[1][0]
        columns_to_drop = ['Unnamed: 0_level_0','Unnamed: 1_level_0','Unnamed: 2_level_0',
                            'Unnamed: 3_level_0','Unnamed: 4_level_0','Unnamed: 5_level_0']
        page[4] = page[4].drop(columns=columns_to_drop, axis=1)
        page[5] = page[5].drop(columns=columns_to_drop, axis=1)
        page[6] = page[6].drop(columns=columns_to_drop, axis=1)
        page[7] = page[7].drop(columns=columns_to_drop, axis=1)
        page[8] = page[8].drop(columns=columns_to_drop, axis=1)
        home_team_stats = pd.concat([page[3], page[4],page[5],page[6],page[7],page[8]], axis=1)
        home_team_stats['Club'] = home_team
        page[11] = page[11].drop(columns=columns_to_drop, axis=1)
        page[12] = page[12].drop(columns=columns_to_drop, axis=1)
        page[13] = page[13].drop(columns=columns_to_drop, axis=1)
        page[14] = page[14].drop(columns=columns_to_drop, axis=1)
        page[15] = page[15].drop(columns=columns_to_drop, axis=1)
        away_team_stats = pd.concat([page[10], page[11],page[12],page[13],page[14],page[15]], axis=1)
        away_team_stats['Club'] = away_team
        both_teams = pd.concat([home_team_stats[:-1],away_team_stats[:-1]], axis=0)
        both_teams = both_teams.reset_index(drop=True)
        both_teams['Season'] = '2020-2021'
        both_teams['MatchNumber'] = i+1
        play_stats_all_games = pd.concat([play_stats_all_games,both_teams], axis=0)
    else:
        print('Table numbers not correct.')

### Reset index of main dataframe

In [18]:
play_stats_all_games = play_stats_all_games.reset_index(drop=True)

### Check all data is there

In [19]:
play_stats_all_games.head(10)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Player                  #             Nation   
0      Roberto Firmino                9.0             br BRA   
1           Sadio Mané               10.0             sn SEN   
2        Mohamed Salah               11.0             eg EGY   
3  Georginio Wijnaldum                5.0             nl NED   
4           Naby Keïta                8.0             gn GUI   
5              Fabinho                3.0             br BRA   
6     Jordan Henderson               14.0            eng ENG   
7         Curtis Jones               17.0            eng ENG   
8     Andrew Robertson               26.0            sco SCO   
9      Virgil van Dijk                4.0             nl NED   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Performance         \
                 Pos                Age                Min         Gls Ast PK   
0                 FW             28-346                 90           0   0  0   
1                 LW             28-155                 90           0   0  0   
2                 RW             28-089                 90           3   0  2   
3                 CM             29-306                 90           0   0  0   
4                 CM             25-215                 57           0   0  0   
5                 CM             26-325                 33           0   0  0   
6                 DM             30-087                 65           0   0  0   
7                 DM             19-226                 25           0   0  0   
8                 LB             26-185                 90           0   1  0   
9                 CB             29-066                 90           1   0  0   

                                                      Expected           SCA  \
  PKatt Sh SoT CrdY CrdR Touches Press Tkl Int Blocks       xG npxG   xA SCA   
0     0  2   0    1    0      49    26   2   0      1      0.5  0.5  0.0   1   
1     0  5   0    0    0      54    20   0   0      2      0.2  0.2  0.8   9   
2     2  7   1    0    0      80    23   3   0      2      1.9  0.4  0.2   8   
3     0  2   1    0    0      37    25   3   0      0      0.4  0.4  0.1   2   
4     0  1   0    0    0      45    24   1   3      2      0.1  0.1  0.0   2   
5     0  0   0    0    0      18     7   1   0      0      0.0  0.0  0.0   1   
6     0  1   1    0    0      43    16   2   0      1      0.0  0.0  0.0   0   
7     0  0   0    0    0      23     8   0   0      2      0.0  0.0  0.0   2   
8     0  0   0    0    0      69     9   0   1      0      0.0  0.0  0.2   1   
9     0  2   1    0    0      42     1   0   0      2      0.2  0.2  0.0   0   

      Passes                   Carries         Dribbles     Total            \
  GCA    Cmp Att  Cmp% PrgDist Carries PrgDist     Succ Att   Cmp Att  Cmp%   
0   0     30  35  85.7     128      31     107        1   1    30  35  85.7   
1   0     32  37  86.5     184      37     142        4   5    32  37  86.5   
2   1     38  49  77.6     211      64     280        9  13    38  49  77.6   
3   0     18  23  78.3      29      21      76        1   3    18  23  78.3   
4   0     25  30  83.3     151      26     109        3   4    25  30  83.3   
5   1     12  14  85.7      65       8      13        0   0    12  14  85.7   
6   0     29  39  74.4     192      24      72        1   1    29  39  74.4   
7   0     15  21  71.4      59      17      51        0   0    15  21  71.4   
8   1     39  60  65.0     416      30      90        0   2    39  60  65.0   
9   0     25  33  75.8     209      19      82        0   0    25  33  75.8   

                  Short            Medium            Long             \
  TotDist PrgDist   Cmp Att   Cmp%    Cmp Att   Cmp%  Cmp Att   Cmp%   
0     397     128    18  20   90.0      5   7   71.4    2   2  100.0   
1     517     184    15  16   93.8     12  13   92.3    2   2  100.0   
2     580     211    18  20   90.0     14  15   93.3    3   8   

### Export to CSV

In [ ]:
play_stats_all_games.to_csv('PlayerGameStats07122020.csv', index=False)